In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pickle
with open(r'C:\Users\bachi\Desktop\IR\Code\embedding_matrix.pkl', 'rb') as f:
    embedding_matrix = pickle.load(f)

In [4]:
import pandas as pd
file_path = r"C:\Users\bachi\Desktop\IR\Data\01 Legal Case Corpus\cases-by-categories\Map.txt"

column_names = ['Case_Type', 'Case_Type_str']

# Load data into a DataFrame
class_names = pd.read_csv(file_path, delimiter='->', names = column_names)

C:\Users\bachi\AppData\Local\Temp\ipykernel_20048\1628909533.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  class_names = pd.read_csv(file_path, delimiter='->', names = column_names)


In [5]:
import pandas as pd

file_path = r"C:\Users\bachi\Desktop\IR\Code\data.csv"

# Load data into a DataFrame
data = pd.read_csv(file_path)

# Display the DataFrame
print(data)

       Unnamed: 0                                   Case_description  \
0               0  Petitioner Jhonatan Acosta (“Acosta”) petition...   
1               1  OPINION Anthony Araujo filed a complaint in th...   
2               2  OPINION Robert Zimmerman was riding his motorc...   
3               3  OPINION This case returns to us after the Supr...   
4               4  OPINION OF THE COURT Haddrick Byrd, a prisoner...   
...           ...                                                ...   
39150       39150  In this claim for benefits under the Black Lun...   
39151       39151  OPINION James Kea petitions for review of the ...   
39152       39152  Mining Energy, Inc., and its insurer, Old Repu...   
39153       39153  CSX Hotels, Incorporated, d/b/a The Greenbrier...   
39154       39154  OPINION On James Martin's application for blac...   

       Case_Type                              processed_description  
0              1  petitioner jhonatan acosta acosta petition cou.

In [6]:
import nltk
import numpy as np
nltk.download('stopwords')
nltk.download('wordnet')
import tkinter as tk
from tkinter import Scrollbar, Text, Entry, Button
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense, LayerNormalization, MultiHeadAttention, Bidirectional

class Classification:
    def __init__(self, vocabulary_size, embedding_dimension, num_classes):
        self.model = Sequential()
        self.model.add(Embedding(input_dim=vocabulary_size, output_dim=embedding_dimension, input_length=200))
        self.model.add(LSTM(150))  # Adjust the number of LSTM units as needed
        self.model.add(Dropout(0.5))
        self.model.add(Dense(num_classes, activation='softmax'))

    def load_weights(self, weights_path):
        self.model.load_weights(weights_path)

    def preprocess_and_reshape(self, user_input):
        if isinstance(user_input, bytes):
            user_input = user_input.decode('utf-8', errors='ignore')        
        tokens = nltk.word_tokenize(user_input)
        stop_words = set(stopwords.words('english'))
        lemmatizer = WordNetLemmatizer()

        # Ensure each token is alphanumeric and not in stop words
        words = [token.lower() for token in tokens if token.isalnum() and token.lower() not in stop_words]

        # Lemmatize each word
        words = [lemmatizer.lemmatize(word) for word in words]
        preprocessed_input = ' '.join(words)  # Fix: use ' '.join() to create a space-separated string
        # Convert words to integers using a mapping or any other method
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts([preprocessed_input])  # Fix: pass a list of strings
        input_tokens = tokenizer.texts_to_sequences([preprocessed_input])  # Fix: pass a list of strings
        padded_input = pad_sequences(input_tokens, maxlen=200, padding='post', truncating='post')
        return padded_input

    def predict(self, input_data):
        input_data = self.preprocess_and_reshape(input_data)
        result = self.model.predict(input_data)
        result = np.argmax(result)
        return result


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bachi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bachi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, LayerNormalization, Dropout, Bidirectional, LSTM
from tensorflow.keras.initializers import Constant

class SentenceGenerationModel(tf.keras.Model):
    def __init__(self, vocabulary_size, embedding_matrix):
        super(SentenceGenerationModel, self).__init__()

        # Create an embedding layer
        self.embedding_layer = Embedding(
            input_dim=vocabulary_size,
            output_dim=embedding_matrix.shape[1],
            embeddings_initializer=Constant(embedding_matrix),
            input_length=200,
            trainable=True
        )

        # Create a TransformerBlock
        self.transformer_block = TransformerBlock(embedding_matrix.shape[1], num_heads=2, ff_dim=32, rnn_units=64)

        # Define the output layer
        self.output_layer = Dense(200, activation='softmax')
        self.model = [self.embedding_layer, self.transformer_block, self.output_layer]
        
        # Build the model
        inputs = Input(shape=(200,))
        embedded_inputs = self.embedding_layer(inputs)
        transformer_output = self.transformer_block(embedded_inputs, training=False)
        outputs = self.output_layer(transformer_output)
        self.model = tf.keras.Model(inputs, outputs)

    def call(self, inputs, training=False):
        return self.model(inputs)
        
    def load_weights(self, weights_path):
        self.model.load_weights(weights_path)

    
    def preprocess_and_reshape(self, user_input):
        print(user_input)
        if isinstance(user_input, bytes):
            user_input = user_input.decode('utf-8', errors='ignore')        
        tokens = nltk.word_tokenize(user_input)
        stop_words = set(stopwords.words('english'))
        lemmatizer = WordNetLemmatizer()

        # Ensure each token is alphanumeric and not in stop words
        words = [token.lower() for token in tokens if token.isalnum() and token.lower() not in stop_words]

        # Lemmatize each word
        words = [lemmatizer.lemmatize(word) for word in words]
        preprocessed_input = ' '.join(words)  # Fix: use ' '.join() to create a space-separated string
        # Convert words to integers using a mapping or any other method
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts([preprocessed_input])  # Fix: pass a list of strings
        input_tokens = tokenizer.texts_to_sequences([preprocessed_input])  # Fix: pass a list of strings
        padded_input = pad_sequences(input_tokens, maxlen=200, padding='post', truncating='post')
        return padded_input
    
    def predict(self, input_data):
        input_data = self.preprocess_and_reshape(input_data)
        # Cast the input data to int32
        #input_data = tf.cast(input_data, tf.int32)
        print("entered to prediction")
        result = self.model.predict(input_data)
        print("result calculated")
        return result


class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rnn_units, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

        # RNN as Encoder
        self.rnn_encoder = Bidirectional(LSTM(units=rnn_units, return_sequences=True))

        # LSTM as Decoder
        self.lstm_decoder = LSTM(units=rnn_units, return_sequences=True)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Instantiate the model

    

In [21]:
class CombinedChatBotModel:
    def __init__(self, classification_model, sentence_model, class_names, data):
        self.model1 = classification_model
        self.model2 = sentence_model
        self.class_names = class_names
        self.data = data

    def chat(self, user_input):
        global output
        global class_label
        class_label = self.model1.predict(user_input)
        # Map the index to the corresponding case type
        for index, case_type in zip(self.class_names['Case_Type'], self.class_names['Case_Type_str']):
            if class_label == index:
                label = case_type
                break
        else:
            label = 'Unknown Case Type'   
        print(label)
        # Predict response using model2
        response = self.model2.predict(user_input)
        # Fetch the top 5 case descriptions for the predicted class label
        output = self.get_top_case_descriptions(class_label)
        
        return label, output
    
    def get_top_case_descriptions(self, class_label, top_k=2):
        # Filter case_data based on the predicted class label
        filtered_cases = self.data[self.data['Case_Type'] == class_label]

        # Get the top 5 case descriptions
        top_case_descriptions = filtered_cases['Case_description'].head(top_k).tolist()

        # Convert the list of case descriptions to a string
        top_case_descriptions_str = "\n".join(top_case_descriptions)

        return top_case_descriptions_str

class ChatBotApp:
    def __init__(self, master, chatbot_model):
        self.master = master
        self.master.title("Legal Bot")

        self.chatbot_model = chatbot_model

        self.create_widgets()

    def create_widgets(self):
        self.chat_display = Text(self.master, wrap=tk.WORD, width=40, height=15)
        self.chat_display.grid(row=0, column=0, padx=10, pady=10, columnspan=2)
        self.chat_display.config(state=tk.DISABLED)

        self.scrollbar = Scrollbar(self.master, command=self.chat_display.yview)
        self.scrollbar.grid(row=0, column=2, sticky='nsew')
        self.chat_display['yscrollcommand'] = self.scrollbar.set

        self.input_entry = Entry(self.master, width=30)
        self.input_entry.grid(row=1, column=0, padx=10, pady=10)

        self.send_button = Button(self.master, text="Send", command=self.send_message)
        self.send_button.grid(row=1, column=1, pady=10)

    def send_message(self):
        user_input = self.input_entry.get()
        self.display_message("User: " + user_input)

        # Call the combined chatbot model to get the response
        class_label, response = self.chatbot_model.chat(user_input)

        self.display_message("Bot: " + "This case belongs to " + str(class_label) + ". Here are few more details. " + str(response))

    def display_message(self, message):
        self.chat_display.config(state=tk.NORMAL)
        self.chat_display.insert(tk.END, message + "\n")
        self.chat_display.config(state=tk.DISABLED)
        self.chat_display.see(tk.END)
def load_and_create_model():
    # Load weights for the classification model
    classification_weights_path = r"C:\Users\bachi\Desktop\IR\Code\model1_weights.h5"
    classification_model = Classification(vocabulary_size=268458, embedding_dimension=200, num_classes=76)
    classification_model.load_weights(classification_weights_path)

    # Load the second model
    model2_path = r"C:\Users\bachi\Desktop\IR\Code\model2_weights.h5-20231201T075337Z-001\model2_weights.h5"
    sentence_model = SentenceGenerationModel(vocabulary_size=268458, embedding_matrix=embedding_matrix)
    sentence_model.load_weights(model2_path)
    # Create CombinedChatBotModel
    chatbot_model = CombinedChatBotModel(classification_model, sentence_model, class_names, data)

    return chatbot_model

if __name__ == "__main__":
    # Load the model and create CombinedChatBotModel instance
    loaded_chatbot_model = load_and_create_model()

    # Create Tkinter instance after loading the model
    root = tk.Tk()
    app = ChatBotApp(root, loaded_chatbot_model)
    root.mainloop()

1/1 [==============================] - 1s 1s/step
 Remedies
I need help with sexual abuse. My  pictures are trafficking all over the internet
entered to prediction
1/1 [==============================] - 1s 639ms/step
result calculated


In [22]:
import pandas as pd

data['Case_description']=data['Case_description'].astype(str)
# Merge all rows of the 'case_description' column into a single text
relevant_cases = data[data['Case_Type'] == class_label]
case_descriptions = relevant_cases['Case_description'].tolist()

#case_descriptions_array = case_descriptions
# Print the merged text
#print(case_descriptions)


In [23]:
from nltk.translate.bleu_score import sentence_bleu

# Select the first 100 sequences as references
#num_references = 100
reference_tokens = [reference.lower().split() for reference in case_descriptions] #[:num_references]]
print("Reference Tokens generated")
generated_tokens = output.split()
print("Target Tokens generated")

# Calculate BLEU score
bleu_score = sentence_bleu(reference_tokens, generated_tokens)
print("BLEU Score:", bleu_score)

Reference Tokens generated
Target Tokens generated
BLEU Score: 0.7424638341447329


In [24]:
print(reference_tokens[50])

['appellant', 'american', 'airlines', '(american)', 'appeals', 'from', 'the', 'judgment', 'entered', 'on', 'verdicts', 'for', 'appellees', 'joe', 'and', 'mary', 'rustenhaven', '(mr.', 'and', 'mrs.', 'rustenhaven).', 'we', 'affirm,', 'conditioned', 'on', 'the', 'acceptance', 'by', 'the', 'rustenhavens', 'of', 'the', 'remittiturs', 'ordered', 'on', 'their', 'respective', 'verdicts.', 'the', 'rustenhavens', 'brought', 'suit', 'for', 'damages', 'resulting', 'from', 'the', 'crash', 'of', "american's", 'flight', '1420', 'in', 'little', 'rock,', 'arkansas,', 'on', 'june', '1,', '1999,', 'for', 'which', 'american', 'admitted', 'liability.1', 'they', 'sought', 'damages', 'for', 'mr.', "rustenhaven's", 'lost', 'past', 'wages,', 'his', 'lost', 'earning', 'capacity,', 'his', 'past', 'and', 'future', 'medical', 'costs,', 'and', 'his', 'pain', 'and', 'suffering', 'and', 'for', 'mrs.', "rustenhaven's", 'loss', 'of', 'consortium.', 'after', 'a', 'four-day', 'trial,', 'the', 'jury', 'returned', 'verdic

In [25]:
print(generated_tokens)

['Title', '28', 'U.', 'S.', 'C.', '§1920,', 'as', 'amended', 'by', 'the', 'Court', 'Interpreters', 'Act,', 'includes', '"compensation', 'of', 'interpreters"', 'among', 'the', 'costs', 'that', 'may', 'be', 'awarded', 'to', 'prevailing', 'parties', 'in', 'federal-court', 'lawsuits.', '§1920(6).', 'In', 'this', 'case,', 'the', 'District', 'Court', 'awarded', 'costs', 'to', 'respondent', 'as', 'the', 'prevailing', 'party', 'in', 'a', 'civil', 'action', 'instituted', 'by', 'petitioner.', 'The', 'award', 'included', 'the', 'cost', 'of', 'translating', 'from', 'Japanese', 'to', 'English', 'certain', 'documents', 'that', 'respondent', 'used', 'in', 'preparing', 'its', 'defense.', 'The', 'Ninth', 'Circuit', 'affirmed,', 'concluding', 'that', '§1920(6)', 'covers', 'the', 'cost', 'of', 'translating', 'documents', 'as', 'well', 'as', 'the', 'cost', 'of', 'translating', 'live', 'speech.', 'Held:', 'Because', 'the', 'ordinary', 'meaning', 'of', '"interpreter"', 'is', 'someone', 'who', 'translates', 